In [1]:
import numpy as np
import pandas as pd
import sys

from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import train_test_split


from lightgbm import LGBMClassifier

/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = np.load('../cache/train_stage2_fe2.npy')
df_test = np.load('../cache/test_stage2_fe2.npy')
df = pd.read_csv('../cache/stage2_labels.csv')
y = df['y'].values

df = pd.read_csv('../cache/stage2_test_id.csv')
pid = df.ID.values

wts_per_class = np.load('../cache/stage2_train_weights_per_class.npy')
wts_per_class = wts_per_class.tolist()
print(wts_per_class)

{1: 4.572507552870091, 2: 6.407630522088353, 3: 37.427083333333336, 4: 3.912117177097204, 5: 12.816479400749063, 6: 11.42087542087542, 7: 2.5, 8: 174.66666666666666, 9: 84.79069767441861}


In [3]:
w = np.array([wts_per_class[j] for j in y],)

In [4]:
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_leaves': 50,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'colsample_bytree': 0.8,
#     'n_estimators': 500,
#     'verbose': 1
# }

params = {'num_leaves':[150,200], 'max_depth':[7,10],
          'learning_rate':[0.05,0.03],'max_bin':[100,200], 'n_estimators': [200]}


lgbm = LGBMClassifier(objective='multiclass', sample_weight=w)



clf = GridSearchCV(lgbm, params, n_jobs=4, 
                   cv=StratifiedKFold(y, 5, True),
                   scoring = 'neg_log_loss',
                   verbose=2, refit=True)



In [5]:
clf.fit(df_train, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150, total=47.1min
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150, total=47.8min
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150, total=48.0min
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=100, max_d

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 384.3min


[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=150, total=51.2min
[CV] learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=150, total=51.4min
[CV] learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=51.3min
[CV] learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=52.1min
[CV] learning_rate=0.03, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=51.5min
[CV] learning_rate=0.03, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=2

[CV]  learning_rate=0.03, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=28.9min
[CV]  learning_rate=0.03, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=26.9min


[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed: 985.0min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 2 ..., 4 1], n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0.0, n_estimators=10, n_jobs=-1,
        num_leaves=31, objective='multiclass', random_state=0,
        reg_alpha=0.0, reg_lambda=0.0,
        sample_weight=array([ 4.57251,  6.40763, ...,  3.91212,  4.57251]),
        silent=True, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'num_leaves': [150, 200], 'max_depth': [7, 10], 'learning_rate': [0.05, 0.03], 'max_bin': [100, 200], 'n_estimators': [200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=2)

In [6]:
print(clf.best_score_)
print(clf.best_params_)

# -1.87263813814
# {'learning_rate': 0.05, 'max_bin': 100, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}
#final multi_log_loss: 1.2407789319265683
# final f1_score: 0.5135869565217391
# final accuracy: 0.5135869565217391
# [[53  1  1 24 11  0  4  0  0]
#  [ 3 11  0  1  0  0 31  0  0]
#  [ 1  0  1  2  0  0  3  0  0]
#  [12  2  0 38  0  2 11  0  0]
#  [ 6  0  0  6  7  5  1  0  0]
#  [ 4  2  0  2  7  1  6  0  0]
#  [ 1 13  1  7  4  0 75  0  0]
#  [ 1  0  0  0  0  0  1  0  0]
#  [ 1  0  0  1  0  0  1  0  3]]

# -0.885334702923
# {'learning_rate': 0.03, 'max_bin': 200, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}
# imp: !!!!!!!!!!!! test labels?????????
# final multi_log_loss: 1.2407789319265683
# final f1_score: 0.5135869565217391
# final accuracy: 0.5135869565217391
# [[53  1  1 24 11  0  4  0  0]
#  [ 3 11  0  1  0  0 31  0  0]
#  [ 1  0  1  2  0  0  3  0  0]
#  [12  2  0 38  0  2 11  0  0]
#  [ 6  0  0  6  7  5  1  0  0]
#  [ 4  2  0  2  7  1  6  0  0]
#  [ 1 13  1  7  4  0 75  0  0]
#  [ 1  0  0  0  0  0  1  0  0]
#  [ 1  0  0  1  0  0  1  0  3]]

#-0.871917791514
# {'learning_rate': 0.03, 'max_bin': 100, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}



-0.871917791514
{'learning_rate': 0.03, 'max_bin': 100, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}


In [11]:
test_preds = clf.predict_proba(df_test)
print(test_preds)

[[ 0.31318263  0.02497605  0.03084461 ...,  0.04175446  0.01697367
   0.03246677]
 [ 0.87676647  0.03387073  0.00731554 ...,  0.01248632  0.01044598
   0.00793319]
 [ 0.08155912  0.120782    0.03508653 ...,  0.29990438  0.08416308
   0.0380386 ]
 ..., 
 [ 0.16690811  0.2039015   0.08834835 ...,  0.05275838  0.02663067
   0.01562058]
 [ 0.05823572  0.16886471  0.02484933 ...,  0.25140667  0.01988973
   0.02358954]
 [ 0.37225815  0.30451184  0.01946348 ...,  0.10202609  0.03557978
   0.02117382]]


In [12]:
test_preds = test_preds.clip(min=0.05, max=0.95) # hack for kaggle since kaggle punishes severly if you predict 
# 1 and it is 0

In [13]:


submission = pd.DataFrame(test_preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
submission.to_csv('../submissions/subm_xgb_stage2_lgbm.csv', index=False)
# 2.62613 on stage2 private LB, 1.77046 on stage2 public LB


In [5]:
x1 = np.load('../cache/train_stage2_x1.npy')
x2 = np.load('../cache/train_stage2_x2.npy')
y1 = np.load('../cache/train_stage2_y1.npy')
y2 = np.load('../cache/train_stage2_y2.npy')

# w1 = np.array([wts_per_class[j] for j in y1], )
# w2 = np.array([wts_per_class[j] for j in y2], )

w1 = np.load('../cache/stage2_train_weights_per_class.npy').tolist()
w2 = np.load('../cache/stage2_train_weights_per_class.npy').tolist()

w1_p = np.array([w1[j] for j in y1], )
w2_p = np.array([w2[j] for j in y2], )

In [6]:
best_params_ = {'learning_rate': 0.03, 'max_bin': 100, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}
# clf1 = LGBMClassifier(** clf.best_params_)
clf1 = LGBMClassifier(** best_params_)

In [8]:
clf1.fit(x1, y1, sample_weight=w1_p, eval_metric='multi_logloss')

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.03,
        max_bin=100, max_depth=7, min_child_samples=10, min_child_weight=5,
        min_split_gain=0.0, n_estimators=200, n_jobs=-1, num_leaves=150,
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1)

In [51]:
# Notes -
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_leaves': 50,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'colsample_bytree': 0.8,
#     'n_estimators': 500,
#     'verbose': 1
# }
# with above params we get
# final multi_log_loss: 1.3155948686089638
# final f1_score: 0.0517503805175038
# final accuracy: 0.04619565217391304
# [[ 0  1 26  7  4  6  0  0  0]
#  [ 6  0  2  0  0 32  0  0  0]
#  [ 0  1  2  0  0  3  0  0  0]
#  [ 4  0 42  0  0  6  0  0  0]
#  [ 0  0  7  1  8  3  0  0  0]
#  [ 2  0  2  4  4  7  0  0  0]
#  [11  4  4  1  1 80  0  0  0]
#  [ 1  0  0  0  0  1  0  0  0]
#  [ 0  0  2  0  0  1  0  3  0]]

# params = {'num_leaves':150, 'objective':'multiclass','max_depth':7,'metric': 'multi_logloss',
#           'learning_rate':.05,'max_bin':200, 'n_estimators': 500}
# x1, x2, y1, y2 = train_test_split(df_train, y, test_size=0.2, random_state=42)
# final multi_log_loss: 1.5149298479876543
# final f1_score: 0.07496251874062969
# final accuracy: 0.06793478260869565
# [[10  2 17  4 13  5  0  0  0]
#  [12  0  4  3  1 22  0  0  0]
#  [ 0  1  1  1  2  2  0  0  0]
#  [ 4  5 37  0  2  4  0  0  0]
#  [ 0  2  5  7  8  1  0  0  0]
#  [ 5  0  1  2  7  4  0  1  0]
#  [25  5  5  4  5 52  2  0  0]
#  [ 1  0  0  0  0  1  0  0  0]
#  [ 0  1  1  1  0  0  0  1  0]]

In [9]:
test_preds = clf1.predict_proba(x2)
print(test_preds)

[[  3.98097263e-03   1.45361353e-03   1.44987537e-03 ...,   1.81148520e-03
    6.51550650e-04   9.83917936e-01]
 [  7.25566393e-03   5.17022718e-01   1.24489887e-02 ...,   4.04780439e-01
    8.39524935e-03   5.28026041e-03]
 [  6.13050334e-02   2.60611772e-02   6.01954447e-03 ...,   6.96236997e-02
    4.16880102e-03   4.18837863e-03]
 ..., 
 [  8.71636363e-02   4.54541879e-03   5.23193718e-03 ...,   4.38509766e-03
    4.03257215e-03   4.03006113e-03]
 [  6.88071590e-03   2.60055707e-03   3.54152727e-03 ...,   1.80902596e-03
    1.35111532e-03   1.31035357e-03]
 [  1.25262168e-02   3.57814879e-01   6.90259009e-03 ...,   5.74976752e-01
    6.36528906e-03   6.16424207e-03]]


In [10]:
test_preds = test_preds.clip(min=0.05, max=0.95)

In [13]:
score2 = log_loss(y2, test_preds, labels = list(range(1,10)))
print('CV multi_log_loss: {}'.format(score2))

fscore = f1_score(y2, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='weighted')
print('CV f1_score: {}'.format(fscore))

acc = accuracy_score(y2, test_preds.argmax(axis=1)+1)
print('CV accuracy: {}'.format(acc))

print(confusion_matrix(y2, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))

# final multi_log_loss: 1.5149298479876543
# final f1_score: 0.07496251874062969
# final accuracy: 0.06793478260869565
# [[10  2 17  4 13  5  0  0  0]
#  [12  0  4  3  1 22  0  0  0]
#  [ 0  1  1  1  2  2  0  0  0]
#  [ 4  5 37  0  2  4  0  0  0]
#  [ 0  2  5  7  8  1  0  0  0]
#  [ 5  0  1  2  7  4  0  1  0]
#  [25  5  5  4  5 52  2  0  0]
#  [ 1  0  0  0  0  1  0  0  0]
#  [ 0  1  1  1  0  0  0  1  0]]

CV multi_log_loss: 1.066500698876787
CV f1_score: 0.6855632734162391
CV accuracy: 0.6842818428184282
[[ 82   3   0  22  19   5   2   0   0]
 [  2  65   0   1   1   0  29   2   0]
 [  1   1  11   4   0   0   2   0   0]
 [ 23   3   1 110   6   1   5   0   1]
 [  9   2   1   4  27   3   7   0   0]
 [  6   4   1   5   2  38   3   0   0]
 [  2  22  11   6   4   0 164   1   1]
 [  0   1   0   0   0   0   2   1   0]
 [  0   0   0   1   0   0   1   0   7]]


In [ ]:
# with stratified split
# CV multi_log_loss: 1.066500698876787
# CV f1_score: 0.6842818428184282, CV f1_score: 0.6117058344203261 (macro), 0.6855632734162391 (weigted)
# CV accuracy: 0.6842818428184282
# [[ 82   3   0  22  19   5   2   0   0]
#  [  2  65   0   1   1   0  29   2   0]
#  [  1   1  11   4   0   0   2   0   0]
#  [ 23   3   1 110   6   1   5   0   1]
#  [  9   2   1   4  27   3   7   0   0]
#  [  6   4   1   5   2  38   3   0   0]
#  [  2  22  11   6   4   0 164   1   1]
#  [  0   1   0   0   0   0   2   1   0]
#  [  0   0   0   1   0   0   1   0   7]]

# without stratified split
# CV multi_log_loss: 0.8681909129878375
# CV f1_score: 0.6964769647696477
# CV accuracy: 0.6964769647696477
# [[ 92   0   3  19  10   1   4   0   0]
#  [  4  56   0   3   0   2  41   0   0]
#  [  0   0  11   5   3   0   5   0   0]
#  [ 17   4   2 110   2   2   4   0   0]
#  [ 12   0   0   1  33   2   5   0   1]
#  [  4   3   1   3   4  37   3   0   0]
#  [  1  34   9   6   1   0 169   0   1]
#  [  0   0   0   0   0   0   1   1   0]
#  [  0   1   0   0   0   0   0   0   5]]